### Variables

In [0]:
ON_COLAB = True
experiment = 'exp02'

In [0]:
if ON_COLAB:
    project_dir = 'drive/My Drive/Colab Notebooks/TextGen'
else:
    project_dir = '..'

### Setup and Imports

In [0]:
# Load the "autoreload" extension so that code can change
%load_ext autoreload

# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import os
import sys

In [0]:
if ON_COLAB:
    %tensorflow_version 2.x

In [5]:
if ON_COLAB:# Allow access to src functions from google drive
    from google.colab import drive
    drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
if ON_COLAB:
    # Install extra requirements (src, socialml)

    # src files (added via path, colab doesn't seem to like installing it via pip)
    src_path = os.path.join(project_dir)
    sys.path.append(src_path)

    # socialml
    !pip install git+https://github.com/erees1/socialml.git#egg=socialml

  Cloning https://github.com/erees1/socialml.git to /tmp/pip-install-cefts3wu/socialml
  Running command git clone -q https://github.com/erees1/socialml.git /tmp/pip-install-cefts3wu/socialml
  Created wheel for socialml: filename=socialml-0.1-cp36-none-any.whl size=6993 sha256=f7e1e0c4e9346e0d97cec41d05bea94bc0e1982e9a9fd82cba2c47107b008fe6
  Stored in directory: /tmp/pip-ephem-wheel-cache-o5g6uwln/wheels/29/78/7e/cae2ac03d8e7733ab60f482721cb57629c6e5788af4c061cef
Successfully built socialml


In [0]:
from src.models import seq2seq, data_pre_processing, train_model
from src.data.word_utils import Vocab
import socialml
from src.data import colab_utils
import sys
import os

### Run training script

In [0]:
# Setup file paths relative to project directory
drive_data_loc = os.path.join(project_dir, 'data/processed')
spec_path = os.path.join(project_dir, 'specs', experiment+'.yaml')
log_dir = os.path.join(project_dir, 'models/logs', experiment)
checkpoint_dir = os.path.join(project_dir, 'models/checkpoints', experiment)
train_script_path = os.path.join(project_dir, 'src/models/train_model.py')

In [0]:
# If on colab copy contents of data/processed from gdrive to local machine (reading directly from gdrive is slow)
if ON_COLAB:
    if not os.path.exists('data/processed'):
        colab_utils.copy_to_local(drive_data_loc, 'data/processed')

# Data files relative to processed data directory
vocab_filepath = os.path.join('data/processed', 'vocab_pp.json')
array_path = os.path.join('data/processed', 'msg_tokens.npz')

In [0]:
# Run with python script
#!python $train_script_path $array_path --spec_path $spec_path --log_dir $log_dir --checkpoint_dir $checkpoint_dir --vocab_filepath $vocab_filepath

In [0]:
# Run using import (works better on colab)
train_model.build_and_train(
    array_path,
    spec_path=spec_path,
    log_dir=log_dir,
    checkpoint_dir=checkpoint_dir,
    vocab_filepath=vocab_filepath
)

spec: {'model_params': {'encoder_depth': 1, 'encoder_units': 512, 'decoder_depth': 1, 'embedding_dim': 128}, 'tf_dataset_params': {'batch_size': 32, 'buffer_size': 10000}, 'training_params': {'epochs': 10}}
Epoch 1/10
4220/4220 [==============================] - 4724s 1s/step - loss: 0.2919
Epoch 2/10
4220/4220 [==============================] - 4731s 1s/step - loss: 0.2584
Epoch 3/10
4220/4220 [==============================] - 4694s 1s/step - loss: 0.2435
Epoch 4/10
4220/4220 [==============================] - 4865s 1s/step - loss: 0.2315
Epoch 5/10
4220/4220 [==============================] - 4868s 1s/step - loss: 0.2207
Epoch 6/10
4220/4220 [==============================] - 4843s 1s/step - loss: 0.2107
Epoch 7/10
4220/4220 [==============================] - 4824s 1s/step - loss: 0.2018
Epoch 8/10
2111/4220 [==============>...............] - ETA: 40:12 - loss: 0.1931